<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.leaky_relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 90:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.078125
Current benign train loss: 2.39605975151062

Current batch: 100
Current benign train accuracy: 0.1953125
Current benign train loss: 2.0257203578948975

Current batch: 200
Current benign train accuracy: 0.265625
Current benign train loss: 1.9625966548919678

Current batch: 300
Current benign train accuracy: 0.2890625
Current benign train loss: 1.8750247955322266

Current batch: 400
Current benign train accuracy: 0.390625
Current benign train loss: 1.71024751663208

Current batch: 500
Current benign train accuracy: 0.3125
Current benign train loss: 1.8626947402954102

Current batch: 600
Current benign train accuracy: 0.421875
Current benign train loss: 1.564581274986267

Current batch: 700
Current benign train accuracy: 0.390625
Current benign train loss: 1.6357799768447876

Current batch: 800
Current benign train accuracy: 0.4296875
Current benign train loss: 1.6010286808013916

Current batch: 900
Current beni


Current batch: 800
Current benign train accuracy: 0.859375
Current benign train loss: 0.4347788691520691

Current batch: 900
Current benign train accuracy: 0.875
Current benign train loss: 0.2826688289642334

Current batch: 1000
Current benign train accuracy: 0.8203125
Current benign train loss: 0.4820750057697296

Current batch: 1100
Current benign train accuracy: 0.859375
Current benign train loss: 0.3846873342990875

Total benign train accuarcy: 85.58733333333333
Total benign train loss: 486.59748461842537

[ Test epoch: 5 ]

Test accuarcy: 83.36
Test average loss: 0.005048474791646004
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.8984375
Current benign train loss: 0.35768938064575195

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.28637608885765076

Current batch: 200
Current benign train accuracy: 0.9140625
Current benign train loss: 0.32760822772979736

Current batch: 300
Current benign train accurac


Current batch: 200
Current benign train accuracy: 0.875
Current benign train loss: 0.2713872790336609

Current batch: 300
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3149804174900055

Current batch: 400
Current benign train accuracy: 0.8515625
Current benign train loss: 0.3984792232513428

Current batch: 500
Current benign train accuracy: 0.875
Current benign train loss: 0.33226674795150757

Current batch: 600
Current benign train accuracy: 0.890625
Current benign train loss: 0.30244898796081543

Current batch: 700
Current benign train accuracy: 0.890625
Current benign train loss: 0.3183141052722931

Current batch: 800
Current benign train accuracy: 0.890625
Current benign train loss: 0.30092406272888184

Current batch: 900
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2694339454174042

Current batch: 1000
Current benign train accuracy: 0.859375
Current benign train loss: 0.3308979272842407

Current batch: 1100
Current benign train acc


Current batch: 1000
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2677585780620575

Current batch: 1100
Current benign train accuracy: 0.953125
Current benign train loss: 0.17285440862178802

Total benign train accuarcy: 91.06266666666667
Total benign train loss: 306.0804117247462

[ Test epoch: 16 ]

Test accuarcy: 83.82
Test average loss: 0.0053063095152378085
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.921875
Current benign train loss: 0.17749221622943878

Current batch: 100
Current benign train accuracy: 0.875
Current benign train loss: 0.3338658809661865

Current batch: 200
Current benign train accuracy: 0.9453125
Current benign train loss: 0.16106697916984558

Current batch: 300
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2563323676586151

Current batch: 400
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2266893833875656

Current batch: 500
Current benign train accur


Current batch: 400
Current benign train accuracy: 0.921875
Current benign train loss: 0.21757422387599945

Current batch: 500
Current benign train accuracy: 0.9140625
Current benign train loss: 0.265396386384964

Current batch: 600
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2378024309873581

Current batch: 700
Current benign train accuracy: 0.9375
Current benign train loss: 0.16940511763095856

Current batch: 800
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3024218678474426

Current batch: 900
Current benign train accuracy: 0.9375
Current benign train loss: 0.23878762125968933

Current batch: 1000
Current benign train accuracy: 0.90625
Current benign train loss: 0.2409530133008957

Current batch: 1100
Current benign train accuracy: 0.921875
Current benign train loss: 0.27903684973716736

Total benign train accuarcy: 91.58066666666667
Total benign train loss: 288.89840906858444

[ Test epoch: 22 ]

Test accuarcy: 84.89
Test average lo

...


Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.011424166150391102

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.010287566110491753

Current batch: 900
Current benign train accuracy: 0.9921875
Current benign train loss: 0.029043635353446007

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.01251504197716713

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.006269360892474651

Total benign train accuarcy: 99.55533333333334
Total benign train loss: 18.21930313159828

[ Test epoch: 72 ]

Test accuarcy: 92.48
Test average loss: 0.0028899101883172987
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.009153724648058414

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.008441676385700703

Current batch: 200
Current benign train accuracy: 0.9921875


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.006103143095970154

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.026715001091361046

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.011196043342351913

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.015238693915307522

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.008544049225747585

Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.027897534891963005

Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.024727141484618187

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.007599943783134222

Current batch: 900
Current benign train accuracy: 0.984375
Current benign train loss: 0.016777902841567993

Current batch: 1000
Current benign train accuracy


Current batch: 900
Current benign train accuracy: 0.9765625
Current benign train loss: 0.07011842727661133

Current batch: 1000
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02846439927816391

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.005208205431699753

Total benign train accuarcy: 99.24266666666666
Total benign train loss: 28.008887917385437

[ Test epoch: 83 ]

Test accuarcy: 91.66
Test average loss: 0.0031507821761071683
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02902119792997837

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.015985649079084396

Current batch: 200
Current benign train accuracy: 0.984375
Current benign train loss: 0.031168222427368164

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.004155059345066547

Current batch: 400
Current benign train acc


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.02256530337035656

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.013398727402091026

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.020553743466734886

Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.01679643616080284

Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.022142184898257256

Current batch: 800
Current benign train accuracy: 0.9921875
Current benign train loss: 0.054199621081352234

Current batch: 900
Current benign train accuracy: 0.96875
Current benign train loss: 0.08074416220188141

Current batch: 1000
Current benign train accuracy: 0.984375
Current benign train loss: 0.051008034497499466

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.004660270642489195

Total benign train accuarcy: 99.06266666


Total benign train accuarcy: 99.83666666666667
Total benign train loss: 7.493529670871794

[ Test epoch: 94 ]

Test accuarcy: 93.38
Test average loss: 0.0025267125304788353
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.00908618327230215

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.009736387059092522

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0018362407572567463

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.005303919780999422

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.00069971609627828

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.008188590407371521

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0029357727617025375

Current batch: 700
Current benign train accuracy: 1.0
Current beni